In [1]:
# Python SQL toolkit and Object Relational Mapper
import numpy as np
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func,inspect
from config import pgkey
from sklearn.neighbors import KNeighborsClassifier
import os

In [2]:
stations = pd.read_csv("../Data/Nice_Ride_2017_Station_Locations.csv")
stations_df = pd.DataFrame(stations)
stations_df.head()

,ID,Number,Name,Latitude,Longitude,Total docks
0,1,30000,100 Main Street SE,44.984892,-93.256551,27
1,2,30001,25th Street & 33rd Ave S,44.957341,-93.223374,15
2,3,30002,Riverside Ave & 23rd Ave S,44.967115,-93.240149,15
3,4,30003,Plymouth Ave N & N Oliver Ave,44.991412,-93.306269,15
4,5,30004,11th Street & Hennepin,44.975340,-93.278690,23


In [3]:
bikes = pd.read_csv("../Data/Nice_ride_trip_history_2017_season.csv", low_memory=False)
bikes_df = pd.DataFrame(bikes)
bikes_df['Start Date'] = pd.to_datetime(bikes_df['Start Date'])
bikes_df.head()

,ID,Start Date,Start Time,Start station,Start station number,End Date,End Time,End station,End station number,Account type,Total duration (Seconds)
0,1,2017-11-05,21:45,Hennepin Ave & S Washington Ave,30184,11/5/17,22:02,Logan Park,30104,Member,1048
1,2,2017-11-05,22:45,Broadway Street N & 4th Street E,30122,11/5/17,22:26,Broadway Street N & 4th Street E,30122,Member,2513
2,3,2017-11-05,23:45,Dale Street & Grand Ave.,30106,11/5/17,22:13,N Milton Street & Summit Ave,30101,Member,1817
3,4,2017-11-05,0:45,Weisman Art Museum,30183,11/5/17,22:05,22nd Ave S & Franklin Ave,30014,Casual,1399
4,5,2017-11-05,1:45,South 2nd Street & 3rd Ave S,30030,11/5/17,21:44,6th Ave SE & University Ave,30088,Member,370


In [4]:
weather = pd.read_csv("../Data/WeatherDailyMinneapolis2017.csv", low_memory=False)
weather_df = pd.DataFrame(weather)
col = weather_df.loc[: , "TMAX":"TMIN"]
weather_df['TAVG'] = col.mean(axis=1)
weather_df.head()


,ID,STATION,NAME,DATE,PRCP,TMAX,TMIN,TAVG
0,1,USC00218450,"UNIVERSITY OF MN ST. PAUL, MN US",1/1/17,0.00,26,18,22.0
1,2,USC00218450,"UNIVERSITY OF MN ST. PAUL, MN US",1/2/17,0.00,31,20,25.5
2,3,USC00218450,"UNIVERSITY OF MN ST. PAUL, MN US",1/3/17,0.01,30,25,27.5
3,4,USC00218450,"UNIVERSITY OF MN ST. PAUL, MN US",1/4/17,0.00,25,-6,9.5
4,5,USC00218450,"UNIVERSITY OF MN ST. PAUL, MN US",1/5/17,0.01,2,-8,-3.0


In [5]:
num_stations = bikes_df['Start station'].nunique()
print(num_stations)

202


In [9]:
weather_clean = weather_df[['DATE', 'PRCP', 'TMAX', 'TMIN', 'TAVG']]
weather_clean = weather_clean.rename(columns={'DATE':'Start Date'})
weather_clean.head()
weather_clean['Start Date']=pd.to_datetime(weather_clean['Start Date'])
BigBikes = pd.merge(bikes_df, weather_clean, on="Start Date", how="outer")
BigBikes.head()

,ID,Start Date,Start Time,Start station,Start station number,End Date,End Time,End station,End station number,Account type,Total duration (Seconds),PRCP,TMAX,TMIN,TAVG
0,1.0,2017-11-05,21:45,Hennepin Ave & S Washington Ave,30184,11/5/17,22:02,Logan Park,30104,Member,1048.0,0.1,41,33,37.0
1,2.0,2017-11-05,22:45,Broadway Street N & 4th Street E,30122,11/5/17,22:26,Broadway Street N & 4th Street E,30122,Member,2513.0,0.1,41,33,37.0
2,3.0,2017-11-05,23:45,Dale Street & Grand Ave.,30106,11/5/17,22:13,N Milton Street & Summit Ave,30101,Member,1817.0,0.1,41,33,37.0
3,4.0,2017-11-05,0:45,Weisman Art Museum,30183,11/5/17,22:05,22nd Ave S & Franklin Ave,30014,Casual,1399.0,0.1,41,33,37.0
4,5.0,2017-11-05,1:45,South 2nd Street & 3rd Ave S,30030,11/5/17,21:44,6th Ave SE & University Ave,30088,Member,370.0,0.1,41,33,37.0


In [63]:
#BigBikes = BigBikes.reset_index()
#BigBikes=BigBikes.set_index('Start Date')
dates = BigBikes['Start Date'].unique()
Accounts = ['Member', 'Casual']
BikesArray = []


for date in dates:
    UserArray = []
    UserArray.append(date)
    Members = BigBikes.loc[(BigBikes['Start Date']==date) & (BigBikes['Account type'] == 'Member') ]
    NumMembers = Members['ID'].count()
    UserArray.append(NumMembers)
    Casual = BigBikes.loc[(BigBikes['Start Date']==date) & (BigBikes['Account type'] == 'Casual') ]
    NumCasual = Casual['ID'].count()
    UserArray.append(NumCasual)
    BikesArray.append(UserArray)
    
    

    

In [64]:

datesDF = pd.DataFrame(BikesArray, columns = ['Date', 'TotalMemberTrips','TotalCasualTrips'])
datesDF.head()

,Date,TotalMemberTrips,TotalCasualTrips
0,2017-11-05,242,40
1,2017-11-04,233,91
2,2017-11-03,487,65
3,2017-11-02,625,40
4,2017-11-01,495,26


In [35]:
DayBikes = BigBikes.resample('D', on='Start Date')
DayBikes['Account type'].value_counts()

Start Date  Account type
2017-04-03  Member           465
            Casual            78
2017-04-04  Member          1082
            Casual           303
2017-04-05  Member          1004
                            ... 
2017-11-03  Casual            65
2017-11-04  Member           233
            Casual            91
2017-11-05  Member           242
            Casual            40
Name: Account type, Length: 436, dtype: int64

In [18]:
Members_Group = BigBikes.groupby(["Account type"])
Members_Group.value counts()

,index,ID,Start Date,Start Time,Start station,Start station number,End Date,End Time,End station,End station number,Total duration (Seconds),PRCP,TMAX,TMIN,TAVG
Account type,,,,,,,,,,,,,,,
Casual,170646,170646,217,24,202,202,217,1439,202,202,12989,46,56,46,82
Inconnu,2,2,2,2,2,2,2,2,2,2,2,1,2,2,2
Member,290070,290070,217,24,202,202,217,1439,202,202,6157,46,56,46,82
